## Transcribing mp3 file to numpy requires the FFmpeg. Follow the steps in this link to have it installed. [(How to install FFmpeg in windows)](https://www.wikihow.com/Install-FFmpeg-on-Windows)

In [ ]:
!pip install transformers streamlit accelerate

In [ ]:
!pip list | findstr requests
!pip install requests==2.27.1

In [ ]:
!pip list | findstr requests

In [1]:
import time
from transformers import  pipeline,AutoModelForSpeechSeq2Seq,AutoProcessor
import torch



c:\Users\wtengguangwa\OneDrive - DXC Production\Desktop\pythonenv\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Add these at the very beginning of your notebook, before any other imports
import os
os.environ['CURL_CA_BUNDLE'] = ''

os.environ["TRANSFORMERS_NO_SSL_VERIFY"] = "1"

# Now your regular imports
import time
from transformers import pipeline, AutoModelForSpeechSeq2Seq, AutoProcessor
import torch

# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Create pipeline with SSL verification disabled
model_id = "openai/whisper-small"
pipe = pipeline("automatic-speech-recognition",
                model_id,
                chunk_length_s=30,
                stride_length_s=5,
                return_timestamps=True,
                device=device,
                model_kwargs={"use_safetensors": True, "verify": False},
                token=False)  # Disable token verification

c:\Users\wtengguangwa\OneDrive - DXC Production\Desktop\pythonenv\myenv\Lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\wtengguangwa\OneDrive - DXC Production\Desktop\pythonenv\myenv\Lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\wtengguangwa\OneDrive - DXC Production\Desktop\pythonenv\myenv\Lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cdn-lfs.hf.co'. Adding certificate verification is strongly advised. See: https://urll

In [ ]:
transcription = pipe(r"./english_audio.mp3" )



formatted_lyrics = ""
for line in transcription['chunks']:
    text = line["text"]
    ts = line["timestamp"]
    formatted_lyrics += f"{ts}-->{text}\n"

print(formatted_lyrics.strip())


## Distil WISPER

In [ ]:
model_id = "distil-whisper/distil-small.en"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, low_cpu_mem_usage=True, use_safetensors=True
)

model.to(device)
processor = AutoProcessor.from_pretrained(model_id)
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    return_timestamps=True,
    chunk_length_s=15,
    device=device,
)

In [ ]:
transcription = pipe(r"F:\Gen-AI-Mini-Projects\audio2text_from_mp3\english_audio.mp3" )



formatted_lyrics = ""
for line in transcription['chunks']:
    text = line["text"]
    ts = line["timestamp"]
    formatted_lyrics += f"{ts}-->{text}\n"

print(formatted_lyrics.strip())